In [1]:
import requests
import torch
from PIL import Image
from io import BytesIO

from diffusers import StableDiffusionImg2ImgPipeline

device = "cuda"
model_id_or_path = "runwayml/stable-diffusion-v1-5"
pipe = StableDiffusionImg2ImgPipeline.from_pretrained(model_id_or_path, torch_dtype=torch.float16)
pipe = pipe.to(device)


Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

downloading https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/39593d5650112b4cc580433f6b0435385882d819/text_encoder/pytorch_model.bin to /media/master/text/cv_data/hf_home/hub/tmpiaqtzlul


Storing https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/39593d5650112b4cc580433f6b0435385882d819/text_encoder/pytorch_model.bin in cache at /media/master/text/cv_data/hf_home/hub/models--runwayml--stable-diffusion-v1-5/blobs/770a47a9ffdcfda0b05506a7888ed714d06131d60267e6cf52765d61cf59fd67
Creating pointer from ../../../blobs/770a47a9ffdcfda0b05506a7888ed714d06131d60267e6cf52765d61cf59fd67 to /media/master/text/cv_data/hf_home/hub/models--runwayml--stable-diffusion-v1-5/snapshots/39593d5650112b4cc580433f6b0435385882d819/text_encoder/pytorch_model.bin
/home/master/.conda/envs/ssdc/lib/python3.10/site-packages/transformers-4.28.1-py3.10.egg/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` 

  0%|          | 0/37 [00:00<?, ?it/s]

In [ ]:

url = "https://raw.githubusercontent.com/CompVis/stable-diffusion/main/assets/stable-samples/img2img/sketch-mountains-input.jpg"

response = requests.get(url)
init_image = Image.open(BytesIO(response.content)).convert("RGB")
init_image = init_image.resize((768, 512))


In [5]:
init_image=Image.open("/media/master/wext/cv_data/kitti-full/data/kitti_depth_completion/testing/image/0000000000.png").convert("RGB")

In [7]:

# prompt = "A fantasy landscape, trending on artstation"
# prompt = "An urban landscape"
prompt_embeds=torch.randn(1,4,768)
images = pipe(
    prompt=prompt, 
    # prompt_embeds=prompt_embeds,
              image=init_image, strength=0.75, guidance_scale=7.5).images
images[0].save("debug/fantasy_landscape.png")

  0%|          | 0/37 [00:00<?, ?it/s]

In [17]:
from diffusers import UNet2DConditionModel, StableDiffusionImg2ImgPipeline, AutoencoderKL
import torch

vae = AutoencoderKL(in_channels=1, out_channels=1)
model_id_or_path = "runwayml/stable-diffusion-v1-5"
pipeline = StableDiffusionImg2ImgPipeline.from_pretrained(
    model_id_or_path, unet=model, vae=vae, #torch_dtype=torch.float16
)

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.


In [51]:
vae = AutoencoderKL(in_channels=3, out_channels=1, latent_channels=1)
unet = UNet2DConditionModel(
    sample_size=64,  # the target image resolution
    in_channels=1,  # the number of input channels, 3 for RGB images
    out_channels=1,  # the number of output channels
    layers_per_block=1,  # how many ResNet layers to use per UNet block
    block_out_channels=(
        128,
        256,
        512,
    ),  # the number of output channels for each UNet block
    cross_attention_dim=512,
    down_block_types=(
        "DownBlock2D",
        "CrossAttnDownBlock2D",  # a ResNet downsampling block with spatial self-attention
        "DownBlock2D",
    ),
    up_block_types=(
        "UpBlock2D",  # a regular ResNet upsampling block
        "CrossAttnUpBlock2D",  # a ResNet upsampling block with spatial self-attention
        "UpBlock2D",
    ),
)
pipeline.vae=vae
pipeline.unet=unet

In [52]:
image = torch.randn(1, 3, 64, 64)#.to(torch.float16)
encoder_hidden_states = torch.randn(1, 1, 512)#.to(torch.float16)
samples=pipeline(
    # batch_size=config.eval_batch_size,
    image=image,
    generator=torch.manual_seed(1),
    num_inference_steps=20,
    output_type="numpy",
    prompt_embeds=encoder_hidden_states,
    negative_prompt_embeds=torch.zeros_like(encoder_hidden_states),
).images

/opt/miniconda3/envs/ssdc/lib/python3.10/site-packages/diffusers/image_processor.py:154: FutureWarning: Passing `image` as torch tensor with value range in [-1,1] is deprecated. The expected value range for image tensor is [0,1] when passing as pytorch tensor or numpy Array. You passed `image` with value range [-4.08251428604126,3.788675546646118]
  warnings.warn(


  0%|          | 0/16 [00:00<?, ?it/s]